In [2]:
from pathlib import Path
import json
import os

from tinydb import TinyDB, Query
from ast import literal_eval
import pprint

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    patients_info = []
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')
        self._db_path = Path(db_path)
        self._db = None
        ## TODO: Implement code

        with open(people_json) as user_file:
            file_contents = user_file.read()
        people_dic = json.loads(file_contents)

        with open(visited_json) as user_file:
            file_contents = user_file.read()
        visited_dic = json.loads(file_contents)

        with open(sites_json) as user_file:
            file_contents = user_file.read()
        sites_dic = json.loads(file_contents)

        with open(measurements_json) as user_file:
            file_contents = user_file.read()
        measurements_dic = json.loads(file_contents)

        for p_key in people_dic:
            visit_ids = []
            visit_info = []
            measurements_info = []
            site_info = []
            for m_key in measurements_dic:
                if measurements_dic[m_key]["person_id"] == p_key:
                    visit_ids.append(measurements_dic[m_key]["visit_id"])
                    measurements_info.append(measurements_dic[m_key])
            
            for v_key in visited_dic:
                if visit_ids.count(visited_dic[v_key]["visit_id"]) > 0:
                    visit_info.append(visited_dic[v_key])

            
            patient_info = (people_dic[p_key])
            patient_info["visits"] = visit_info

            for i, visit in enumerate(patient_info["visits"]):
                patient_info["visits"][i]['site'] = sites_dic[visit["site_id"]]
                patient_info["visits"][i]['measurements'] = []
                for m_key in measurements_dic:
                    if measurements_dic[m_key]["visit_id"] == patient_info["visits"][i]['visit_id'] and measurements_dic[m_key]["person_id"] == p_key:
                        patient_info["visits"][i]['measurements'].append(measurements_dic[m_key])

            self.patients_info.append(patient_info)
        # with open(db_path, "w") as outfile:
        #     json.dump(self.patients_info, outfile, indent=2)

        self._load_db()
    
    def _load_db(self):
        self._db = TinyDB(self._db_path)
        self._db.insert_multiple(self.patients_info)
        self._db.close()

        # Used this to test DB with a query
        # from tinydb import Query
        # Todo = Query()
        # self._db = TinyDB(self._db_path)
        # print(self._db.search(Todo.person_id == "dyer"))
        # self._db.close()
        ## TODO: Implement code

In [3]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)
